# Використання моделі `noobai-XL-1.1` для Генерації Зображень у Google Colab

У цьому ноутбуці ми налаштуємо середовище для використання моделі `noobai-XL-1.1` з Hugging Face для завдання генерації зображень на основі текстових описів. Ми завантажимо модель, налаштуємо параметри генерації та продемонструємо приклади використання.

## Встановлення Необхідних Бібліотек

Перш за все, встановимо всі необхідні бібліотеки, які потрібні для роботи з моделлю та генерації зображень.

In [ ]:
# Встановлення необхідних бібліотек
!pip install --upgrade diffusers transformers torch torchvision matplotlib safetensors

## Імпорт Необхідних Модулів

Імпортуємо бібліотеки, необхідні для роботи з моделлю та обробки зображень.

In [ ]:
# Імпорт необхідних модулів
from diffusers import StableDiffusionXLPipeline
from transformers import CLIPTextModel, CLIPTokenizer
import torch
from PIL import Image
import matplotlib.pyplot as plt

## Завантаження Моделі для Генерації Зображень

Завантажимо модель `Laxhar/noobai-XL-1.1` за допомогою `StableDiffusionXLPipeline` з бібліотеки `diffusers`. Переконайтесь, що у вас є доступ до цієї моделі на Hugging Face.

In [ ]:
# Визначення ідентифікатора моделі
model_id = "Laxhar/noobai-XL-1.1"

# Ініціалізація пайплайна
try:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16
    )
    pipe = pipe.to("cuda")  # Переміщення пайплайна на GPU для прискорення
    print("Пайплайн успішно завантажено і переміщено на GPU.")
except Exception as e:
    print(f"Не вдалося завантажити пайплайн: {e}")

## Ініціалізація Токенізатора та Текстових Енкодерів

Для генерації зображень нам потрібні токенізатор та текстові енкодери, які будуть перетворювати текстовий промпт у формат, зрозумілий моделі.

In [ ]:
# Ініціалізація токенізатора та текстових енкодерів
try:
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
    text_encoder_2 = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
    tokenizer = tokenizer.to("cuda")
    text_encoder = text_encoder.to("cuda")
    text_encoder_2 = text_encoder_2.to("cuda")
    print("Токенізатор та текстові енкодери успішно завантажено і переміщено на GPU.")
except Exception as e:
    print(f"Не вдалося завантажити токенізатор або текстові енкодери: {e}")

## Функція для Генерації Зображень

Створимо функцію, яка приймає текстовий опис і генерує відповідне зображення за допомогою моделі.

In [ ]:
def generate_image(prompt, guidance_scale=7.5, num_inference_steps=50):
    """
    Генерує зображення на основі текстового опису.
    
    Parameters:
    - prompt (str): Текстовий опис зображення.
    - guidance_scale (float): Контролює вплив текстового опису на зображення. Вищі значення роблять зображення більш відповідним до опису.
    - num_inference_steps (int): Кількість кроків генерації. Більше кроків можуть покращити якість, але збільшують час генерації.
    
    Returns:
    - image (PIL.Image.Image): Згенероване зображення.
    """
    try:
        # Токенізація промпту
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        
        # Генерація текстових ембеддінгів за допомогою обох енкодерів
        text_embeds1 = text_encoder(**inputs).last_hidden_state
        text_embeds2 = text_encoder_2(**inputs).last_hidden_state
        
        # Генерація зображення з передачею текстових ембеддінгів
        output = pipe(
            prompt=prompt,
            text_embeds=(text_embeds1, text_embeds2),
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps
        )
        
        # Отримання першого згенерованого зображення
        image = output.images[0]
        return image
    except TypeError as e:
        print(f"Виникла помилка TypeError: {e}")
        print("Переконайтесь, що всі необхідні аргументи передані правильно і сумісні з конфігурацією пайплайна.")
        return None
    except Exception as e:
        print(f"Виникла несподівана помилка: {e}")
        return None

## Приклад Використання Функції для Генерації Зображення

Нижче наведено приклад того, як можна використовувати функцію `generate_image` для створення зображення на основі заданого промпту. Ви можете налаштовувати різні параметри генерації за вашими потребами.

In [ ]:
# Визначення промпту
prompt = "A serene sunset over the mountains with a lake in the foreground, ultra-realistic"

# Генерація зображення
generated_image = generate_image(prompt, guidance_scale=10.0, num_inference_steps=100)

# Відображення згенерованого зображення
if generated_image:
    plt.figure(figsize=(10, 10))
    plt.imshow(generated_image)
    plt.axis('off')
    plt.show()
else:
    print("Не вдалося згенерувати зображення.")

## Налаштування Параметрів Генерації

Ви можете експериментувати з різними параметрами генерації, щоб отримати бажані результати. Нижче наведено два приклади з різними параметрами `guidance_scale` та `num_inference_steps`.

In [ ]:
# Визначення нового промпту
prompt = "A futuristic city skyline at night with neon lights and flying cars"

# Висока точність відповідності тексту
generated_image_high_guidance = generate_image(prompt, guidance_scale=10.0, num_inference_steps=100)

# Низька точність відповідності тексту, більш творчий результат
generated_image_low_guidance = generate_image(prompt, guidance_scale=5.0, num_inference_steps=50)

# Відображення зображень
plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.imshow(generated_image_high_guidance)
plt.title("Висока точність відповідності")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(generated_image_low_guidance)
plt.title("Низька точність відповідності")
plt.axis('off')

plt.show()

## Збереження Згенерованих Зображень

Ви можете зберегти згенеровані зображення на свій локальний комп'ютер або в Google Drive.

In [ ]:
# Збереження зображення на локальний диск
generated_image.save("generated_image.png")

# Якщо використовуєте Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Збереження у Google Drive
generated_image.save("/content/drive/MyDrive/generated_image.png")

# Аналогічно для інших згенерованих зображень
generated_image_high_guidance.save("/content/drive/MyDrive/generated_image_high_guidance.png")
generated_image_low_guidance.save("/content/drive/MyDrive/generated_image_low_guidance.png")

## Додаткові Рекомендації

1. **Активуйте GPU у Google Colab:**
   - Перейдіть до `Runtime` -> `Change runtime type`.
   - Виберіть `GPU` як апаратний прискорювач.
   - Натисніть `Save`.

2. **Перевірте Обмеження Моделі:**
   - Деякі моделі можуть мати обмеження на використання або вимагати спеціальних налаштувань. Перевірте [сторінку моделі на Hugging Face](https://huggingface.co/Laxhar/noobai-XL-1.1) для отримання додаткової інформації.

3. **Оптимізація Часу Генерації:**
   - Використання меншої кількості кроків (`num_inference_steps`) або зниження `guidance_scale` може зменшити час генерації, але може вплинути на якість.

4. **Збереження Зображень:**
   - Ви можете зберегти згенеровані зображення на свій локальний комп'ютер або в Google Drive для подальшого використання.